# Training a GAP model

## steps
    1. generate a small dataset of water structures 
        - use CP2K if you havea access to it
        - otherwise: use any simple potential implemented in ASE, just for trying this out I have used EMT here
    1. generate e0 values
    1. separate a training and a validation dataset
    1. **train the model**
    1. look at the outcome of the model
    
## here we will fit twice, to see the difference between a 2b-only and a 2b+3b fit

In [3]:
# general imports 
import numpy as np
import matplotlib.pyplot as plt 
from copy import deepcopy as cp

# ase imports
import ase.io
from ase import Atoms, Atom
from ase import units
from ase.build import molecule
# for MD
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory

In [4]:
# helper functions
def make_water(density, super_cell=[3, 3, 3]):
    """ Geenrates a supercell of water molecules with a desired density.
        Density in g/cm^3!!!"""
    h2o = molecule('H2O')
    a = np.cbrt((sum(h2o.get_masses()) * units.m ** 3 * 1E-6 ) / (density * units.mol))
    h2o.set_cell((a, a, a))
    h2o.set_pbc((True, True, True))
    #return cp(h2o.repeat(super_cell))
    return h2o.repeat(super_cell)

def rms_dict(x_ref, x_pred):
    """ Takes two datasets of the same shape and returns a dictionary containing RMS error data"""

    x_ref = np.array(x_ref)
    x_pred = np.array(x_pred)

    if np.shape(x_pred) != np.shape(x_ref):
        raise ValueError('WARNING: not matching shapes in rms')

    error_2 = (x_ref - x_pred) ** 2

    average = np.sqrt(np.average(error_2))
    std_ = np.sqrt(np.var(error_2))

    return {'rmse': average, 'std': std_}

## generating data only with ASE, using the EMT calculator

This is only for the demonstration of how to do it, this run is will be done very fast. There is no practical use of the data beyond lerning the use teach_sparse, quip, etc. with it.

In [5]:
# Running MD with ASE's EMT

from ase.calculators.emt import EMT
calc = EMT()

T = 150  # Kelvin

# Set up a grid of water
water = make_water(1.0, [3, 3, 3])
water.set_calculator(calc)

# We want to run MD using the Langevin algorithm
# with a time step of 1 fs, the temperature T and the friction
# coefficient to 0.002 atomic units.
dyn = Langevin(water, 1 * units.fs, T * units.kB, 0.0002)

def printenergy(a=water):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

dyn.attach(printenergy, interval=5)

# We also want to save the positions of all atoms after every 5th time step.
traj = Trajectory('dyn_emt.traj', 'w', water)
dyn.attach(traj.write, interval=5)

# Now run the dynamics
printenergy(water)
dyn.run(600)   # CHANGE THIS IF YOU WANT LONGER/SHORTER RUN

Energy per atom: Epot = 0.885eV  Ekin = 0.000eV (T=  0K)  Etot = 0.885eV
Energy per atom: Epot = 0.885eV  Ekin = 0.000eV (T=  0K)  Etot = 0.885eV
Energy per atom: Epot = 0.820eV  Ekin = 0.053eV (T=413K)  Etot = 0.874eV
Energy per atom: Epot = 0.660eV  Ekin = 0.208eV (T=1610K)  Etot = 0.868eV
Energy per atom: Epot = 0.632eV  Ekin = 0.224eV (T=1734K)  Etot = 0.857eV
Energy per atom: Epot = 0.869eV  Ekin = 0.005eV (T= 39K)  Etot = 0.874eV
Energy per atom: Epot = 0.780eV  Ekin = 0.089eV (T=690K)  Etot = 0.869eV
Energy per atom: Epot = 0.755eV  Ekin = 0.117eV (T=909K)  Etot = 0.872eV
Energy per atom: Epot = 0.707eV  Ekin = 0.163eV (T=1262K)  Etot = 0.870eV
Energy per atom: Epot = 0.704eV  Ekin = 0.158eV (T=1220K)  Etot = 0.861eV
Energy per atom: Epot = 0.867eV  Ekin = 0.007eV (T= 54K)  Etot = 0.874eV
Energy per atom: Epot = 0.706eV  Ekin = 0.159eV (T=1228K)  Etot = 0.865eV
Energy per atom: Epot = 0.680eV  Ekin = 0.188eV (T=1458K)  Etot = 0.869eV
Energy per atom: Epot = 0.826eV  Ekin = 0.046

Energy per atom: Epot = 0.352eV  Ekin = 0.597eV (T=4616K)  Etot = 0.948eV
Energy per atom: Epot = 0.395eV  Ekin = 0.548eV (T=4237K)  Etot = 0.943eV
Energy per atom: Epot = 0.335eV  Ekin = 0.608eV (T=4707K)  Etot = 0.944eV
Energy per atom: Epot = 0.439eV  Ekin = 0.525eV (T=4061K)  Etot = 0.964eV
Energy per atom: Epot = 0.416eV  Ekin = 0.547eV (T=4235K)  Etot = 0.963eV
Energy per atom: Epot = 0.414eV  Ekin = 0.554eV (T=4284K)  Etot = 0.968eV
Energy per atom: Epot = 0.392eV  Ekin = 0.576eV (T=4457K)  Etot = 0.968eV
Energy per atom: Epot = 0.379eV  Ekin = 0.585eV (T=4529K)  Etot = 0.965eV
Energy per atom: Epot = 0.374eV  Ekin = 0.593eV (T=4590K)  Etot = 0.967eV
Energy per atom: Epot = 0.387eV  Ekin = 0.582eV (T=4499K)  Etot = 0.969eV
Energy per atom: Epot = 0.357eV  Ekin = 0.612eV (T=4738K)  Etot = 0.970eV


True

In [6]:
# wrap and save traj in .xyz --- the .traj is a non human readable database file, xyz is much better
out_traj = ase.io.read('dyn_emt.traj', ':')
for at in out_traj:
    at.wrap()
    if 'momenta' in at.arrays: del at.arrays['momenta']
ase.io.write('dyn_emt.xyz', out_traj, 'xyz')

## get e0 for H and O - energies of the isolated atoms

This is the energy of the isolated atom, will be in the teach_sparse string in the following format: `e0={H:energy:O:energy}`

In [7]:
isolated_H = Atoms('H', calculator=EMT(), cell=[20, 20, 20], pbc=True)
isolated_O = Atoms('O', calculator=EMT(), cell=[20, 20, 20], pbc=True)

print('e0_H:',isolated_H.get_potential_energy())
print('e0_O:',isolated_O.get_potential_energy())

# this made the e0 string be the following: e0={H:3.21:O:4.6}
isolated_H
for i in isolated_H:
    print(i)

e0_H: 3.21
e0_O: 4.6
Atom('H', [0.0, 0.0, 0.0], index=0)


## separate the dataset into a training and a validation set

As we have 120 frames from the 600fs MD, I will do it 60,60 with taking even and odd frames for the two

In [8]:
ase.io.write('train.xyz', out_traj[0::2] + [isolated_H] + [isolated_O]) 
ase.io.write('validate.xyz', out_traj[1::2])

In [ ]:
# Split up into train and test data. 

out_traj_hydrogen = ase.io.read('/Users/simon/simon_ml/GAP/hydrogen_md.xyz', ':')
for at in out_traj_hydrogen:
    at.wrap()

ase.io.write('train2b_01.xyz',out_traj_hydrogen[0:-1:2] + [isolated_H])
ase.io.write('test2b_01.xyz', out_traj_hydrogen[1::2])

train_data='/Users/simon/simon_ml/GAP/train2b_01.xyz'
test_data='/Users/simon/simon_ml/GAP/test2b_01.xyz'


In [ ]:
# Compare files of the tutorial with selfmade ones to find differences
print(type(out_traj[-1]))
print(type(out_traj_hydrogen[-1]))
train_data

## train our GAP model from the command line

Will use a fit of 2b only, using the desciptor distance_2b.

Let's understand how this works. The bash command takes named arguments separated by spaces.

- `teach_sparse` the command which actually does the fit  
- `e0={H:3.21:O:4.6}` the energies of the isolated atoms
- `energy_parameter_name=energy force_parameter_name=forces` names of the parameters
- `do_copy_at_file=F sparse_separate_file=T` just needed, don't want to copy the training data and using separate files for the xml makes it faster
- `gp_file=GAP.xml` filename of the potential parameters, I have always used this name, because I had separate directories for the different trainings potentials
- `at_file=train.xyz` training file
- `default_sigma={0.008 0.04 0 0}` sigma values to be used for energies, forces, stresses, hessians in order; this represents the accuracy of the data and the relative weight of them in the fit (more accurate --> more significant in the fit)
- `gap={...}` the potential to be fit, separated by ':'

**distance_2b**
- `cutoff=4.0` radial, practically the highest distance the descriptor takes into account 
- `covariance_type=ard_se` use gausses in the fit
- `delta=0.5` what relative portion of the things shall be determined by this potential
- `theta_uniform=1.0` width of the gaussians
- `sparse_method=uniform` use uniform bins to choose the sparse points
- `add_species=T ` take the species into account, so it will generate more GAPs automatically (see the output)
- `n_sparse=10` number of sparse points


## notice, that the script is running in parallel, using all 8 cores of the current machine

In [ ]:
# train gap_model with our hydrogen model
! gap_fit energy_parameter_name=energy force_parameter_name=forces do_copy_at_file=F sparse_separate_file=T gp_file=GAP2b_01.xml at_file='/Users/simon/simon_ml/GAP/train2b_01.xyz' default_sigma={0.008 0.04 0 0} gap={distance_2b cutoff=2.0 covariance_type=ard_se delta=0.5 theta_uniform=1.0 sparse_method=uniform add_species=T n_sparse=10}

## use the potential with QUIP on train.xyz and validate.xyz

In [ ]:
# calculate train.xyz

! quip E=T F=T atoms_filename=/Users/simon/simon_ml/GAP/train2b_01.xyz param_filename=GAP2b_01.xml | grep AT | sed 's/AT//' > quip_train2b_01.xyz
! quip E=T F=T atoms_filename=/Users/simon/simon_ml/GAP/test2b_01.xyz param_filename=GAP2b_01.xml | grep AT | sed 's/AT//' > quip_test2b_01.xyz

## make simple plots of the energies and forces on the EMT and GAP datas

In [ ]:
def energy_plot(in_file, out_file, ax, title='Plot of energy'):
    """ Plots the distribution of energy per atom on the output vs the input"""
    # read files
    in_atoms = ase.io.read(in_file, ':-1')
    out_atoms = ase.io.read(out_file, ':-1')
    # list energies
    ener_in = [at.get_potential_energy() for at in in_atoms]
    ener_out = [at.get_potential_energy()  for at in out_atoms]
    # scatter plot of the data
    ax.scatter(ener_in, ener_out)
    # get the appropriate limits for the plot
    for_limits = np.array(ener_in +ener_out)   
    elim = (for_limits.min() , for_limits.max() )
    ax.set_xlim(elim)
    ax.set_ylim(elim)
    # add line of slope 1 for refrence
    ax.plot(elim, elim, c='k')
    # set labels
    ax.set_ylabel('energy by GAP / eV')
    ax.set_xlabel('energy by EMT / eV')
    #set title
    ax.set_title(title)
    # add text about RMSE
    _rms = rms_dict(ener_in, ener_out)
    rmse_text = 'RMSE:\n' + str(np.round(_rms['rmse'], 3)) + ' +- ' + str(np.round(_rms['std'], 3)) + 'eV/atom'
    ax.text(0.9, 0.1, rmse_text, transform=ax.transAxes, fontsize='large', horizontalalignment='right', 
            verticalalignment='bottom')
    
def force_plot(in_file, out_file, ax, symbol='HO', title='Plot of force'):
    """ Plots the distribution of firce components per atom on the output vs the input 
        only plots for the given atom type(s)"""
    
    in_atoms = ase.io.read(in_file, ':')
    out_atoms = ase.io.read(out_file, ':')
    
    # extract data for only one species
    in_force, out_force = [], []
    for at_in, at_out in zip(in_atoms, out_atoms):
        # get the symbols
        sym_all = at_in.get_chemical_symbols()
        # add force for each atom
        for j, sym in enumerate(sym_all):
            if sym in symbol:
                in_force.append(at_in.get_forces()[j])
                #out_force.append(at_out.get_forces()[j]) \  
                out_force.append(at_out.arrays['force'][j]) # because QUIP and ASE use different names
    # convert to np arrays, much easier to work with
    #in_force = np.array(in_force)
    #out_force = np.array(out_force)
    # scatter plot of the data
    ax.scatter(in_force, out_force)
    # get the appropriate limits for the plot
    for_limits = np.array(in_force + out_force)   
    flim = (for_limits.min() , for_limits.max())
    ax.set_xlim(flim)
    ax.set_ylim(flim)
    # add line of 
    ax.plot(flim, flim, c='k')
    # set labels
    ax.set_ylabel('force by GAP / (eV/Å)')
    ax.set_xlabel('force by EMT / (eV/Å)')
    #set title
    ax.set_title(title)
    # add text about RMSE
    _rms = rms_dict(in_force, out_force)
    rmse_text = 'RMSE:\n' + str(np.round(_rms['rmse'], 3)) + ' +- ' + str(np.round(_rms['std'], 3)) + 'eV/Å'
    ax.text(0.9, 0.1, rmse_text, transform=ax.transAxes, fontsize='large', horizontalalignment='right', 
            verticalalignment='bottom')

In [ ]:
fig, ax_list = plt.subplots(nrows=2, ncols=2, gridspec_kw={'hspace': 0.3})
fig.set_size_inches(15, 20)
ax_list = ax_list.flat[:]

energy_plot('train2b_01.xyz','quip_train2b_01.xyz', ax_list[0], 'Energy on training data')
energy_plot('test2b_01.xyz', 'quip_test2b_01.xyz', ax_list[1], 'Energy on validation data')
force_plot('train2b_01.xyz', 'quip_train2b_01.xyz', ax_list[2], 'H', 'Force on training data - H')
force_plot('test2b_01.xyz', 'quip_test2b_01.xyz', ax_list[3], 'H', 'Force on test data - H')
#force_plot('validate.xyz', 'quip_validate.xyz', ax_list[4], 'H', 'Force on validation data - H')
#force_plot('validate.xyz', 'quip_validate.xyz', ax_list[5], 'O',  'Force on validation data - O')

# if you wanted to have the same limits on the force plots
#for ax in ax_list[2:]:
#    flim = (-20, 20)
#    ax.set_xlim(flim)
#    ax.set_ylim(flim)

## train our GAP_3b model from the command line

Let's add three ody terms to the fit, which will hopefully improve it. We will be using the desciprtors distance_2b and angle_3b.

**angle_3b**
- `theta_fac=0.5` this takes the input data and determines the width from that; useful here, because the dimensions of the descriptor are different
- `n_sparse=50` higher dimensional space, more sparse points

## both training and quip takes significantly more time than the last one!!!

In [ ]:
! gap_fit energy_parameter_name=energy force_parameter_name=forces do_copy_at_file=F sparse_separate_file=T gp_file=GAP2b3b_01.xml at_file=/Users/simon/simon_ml/GAP/train2b_01.xyz default_sigma={0.008 0.04 0 0} gap={distance_2b cutoff=4.0 covariance_type=ard_se delta=0.5 theta_uniform=1.0 sparse_method=uniform add_species=T n_sparse=10 : angle_3b cutoff=3.5 covariance_type=ard_se delta=0.5 theta_fac=0.5 add_species=T n_sparse=30 sparse_method=uniform}


## use the potential with QUIP on trani.xyz and validate.xyz

In [ ]:
# calculate train.xyz

! quip E=T F=T atoms_filename=/Users/simon/simon_ml/GAP/train2b_01.xyz param_filename=GAP2b3b_01.xml | grep AT | sed 's/AT//' > quip3b_train01.xyz
! quip E=T F=T atoms_filename=/Users/simon/simon_ml/GAP/test2b_01.xyz param_filename=GAP2b3b_01.xml | grep AT | sed 's/AT//' > quip3b_test01.xyz

## look at the outputs - clear improvement

In [ ]:
fig, ax_list = plt.subplots(nrows=3, ncols=2, gridspec_kw={'hspace': 0.3})
fig.set_size_inches(15, 20)
ax_list = ax_list.flat[:]

energy_plot('train2b_01.xyz', 'quip3b_train01.xyz', ax_list[0], 'Energy on training data')
energy_plot('test2b_01.xyz', 'quip3b_test01.xyz', ax_list[1], 'Energy on validation data')
#force_plot('train.xyz', 'quip_3b_train.xyz', ax_list[2], 'H', 'Force on training data - H')
#force_plot('train.xyz', 'quip_3b_train.xyz', ax_list[3], 'O', 'Force on training data - O')
#force_plot('validate.xyz', 'quip_3b_validate.xyz', ax_list[4], 'H', 'Force on validation data - H')
#force_plot('validate.xyz', 'quip_3b_validate.xyz', ax_list[5], 'O',  'Force on validation data - O')

# if you wanted to have the same limits on the force plots
#for ax in ax_list[2:]:
#    flim = (-20, 20)
#    ax.set_xlim(flim)
#    ax.set_ylim(flim)